# EPA AQI Data Acquisition

## Acknowledgement

This notebook contains code from the [US EPA Air Quality System API Example](https://drive.google.com/file/d/1B7AGlaW7d-27bHKLVXGBwLt8T-Elx-HB/view?usp=drive_link) licensed under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.2 - August 16, 2024

Data obtained in this code in obtained via the US Environmental Protection Agency (EPA) Air Quality Service (AQS) API. This is a historical API and does not provide real-time air quality data. The [documentation](https://aqs.epa.gov/aqsweb/documents/data_api.html) for the API provides definitions of the different call parameter and examples of the various calls that can be made to the API.

I have utilized the parts of the API Example to do the following:
 - Constants, with the exception of updating city locations to include my assigned city (memphis) 
 - Make Sign-up Request, List Request, Daily Summary Request

In [1]:
# 
#    These are standard python modules
#
#import json, time, urllib.parse
import json, time
#
#    The 'requests' module is a distribution module for making web requests. If you do not have it already, you'll need to install it
import requests
import pandas as pd

/var/folders/js/_m5r698x29j4dvr0w18h6jdm0000gn/T/ipykernel_2912/4071512226.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('intermediate_file_paths.json') as output_path_file:
    output_paths = json.load(output_path_file)


EPA_AQI_OUTPUT_PATH = output_paths.get('stage0_epa_aqi_csv')

In [3]:
#########
#
#    CONSTANTS
#
CITY_LOCATIONS = {

    'memphis': {'city': 'Memphis',
        'county': 'Shelby',
        'state': 'Tennessee',
        'fips': '47157',
        'latlon': [35.11, -89.97]},  # Decimal degrees (35.11, -89.97
    
}
#
#    This is the root of all AQS API URLs
#
API_REQUEST_URL = 'https://aqs.epa.gov/data/api'

#
#    These are some of the 'actions' we can ask the API to take or requests that we can make of the API
#
#    Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'
#
#    List actions provide information on API parameter values that are required by some other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'
#
#    Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '/monitors/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_MONITORS_BOX = '/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '/dailyData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_DAILY_SUMMARY_BOX = '/dailyData/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'

API_ACTION_ANNUAL_SUMMARY_COUNTY = '/annualData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'



#
#    It is always nice to be respectful of a free data resource.
#    We're going to observe a 100 requests per minute limit - which is fairly nice
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED
#
#
#    This is a template that covers most of the parameters for the actions we might take, from the set of actions
#    above. In the examples below, most of the time parameters can either be supplied as individual values to a
#    function - or they can be set in a copy of the template and passed in with the template.
# 
AQS_REQUEST_TEMPLATE = {
    "email":      "",     
    "key":        "",      
    "state":      "",     # the two digit state FIPS # as a string
    "county":     "",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    "end_date":   "",     # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}



## Making a sign-up request

(from US EPA Air Quality System API Example)

Before you can use the API you need to request a key. You will use an email address to make the request. The EPA then sends a confirmation email link and a 'key' that you use for all other requests.

You only need to sign-up once, unless you want to invalidate your current key (by getting a new key) or you lose your key.


In [ ]:
#
#    This implements the sign-up request. The parameters are standardized so that this function definition matches
#    all of the others. However, the easiest way to call this is to simply call this function with your preferred
#    email address.
#
def request_signup(email_address = None,
                   endpoint_url = API_REQUEST_URL, 
                   endpoint_action = API_ACTION_SIGNUP, 
                   request_template = AQS_REQUEST_TEMPLATE,
                   headers = None):
    
    # Make sure we have a string - if you don't have access to this email addres, things might go badly for you
    if email_address:
        request_template['email'] = email_address        
    
    if not request_template['email']: 
        raise Exception("Must supply an email address to call 'request_signup()'")

    if '@' not in request_template['email']: 
        raise Exception(f"Must supply an email address to call 'request_signup()'. The string '{request_template['email']}' does not look like an email address.")

    # Compose the signup url - create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [4]:

#    A SIGNUP request is only to be done once, to request a key. A key is sent to that email address and needs to be confirmed with a click through
#    This code should probably be commented out after you've made your key request to make sure you don't accidentally make a new sign-up request

# print("Requesting SIGNUP ...")
# USERNAME = "djsdavis@uw.edu"
# response = request_signup(USERNAME)
# print(json.dumps(response,indent=4))


## Get Username and API Key for API Authentication

This is my own code. Not taken from the US EPA Air Quality System API Example

Note: config.json is included in the .gitignore file. You will need to add your own proj_config.json file with this format if you want to use the code below:

```
{
  "api_username": "youremailaddress@something.com",
  "apikey": "yourapikey"
}

```

In [4]:
# Load the config file
with open('proj_config.json') as config_file:
    config = json.load(config_file)

# Get the username and access token
username_api_auth = config.get('api_username')

if not username_api_auth:
    raise Exception("API email address not found in config file")

api_key = config.get('apikey')

if not api_key:
    raise Exception("API key not found in config file")


USERNAME = username_api_auth
APIKEY = api_key

## Making a list request
(from US EPA Air Quality System API Example)

Once you have a key, the next thing is to get information about the different types of air quality monitoring (sensors) and the different places where we might find air quality stations. The monitoring system is complex and changes all the time. The EPA implementation allows an API user to find changes to monitoring sites and sensors by making requests - maybe monthly, or daily. This API approach is probably better than having the EPA publish documentation that may be out of date as soon as it hits a web page. The one problem here is that some of the responses rely on jargon or terms-of-art. That is, one needs to know a bit about the way atmospheric sciece works to understand some of the terms. ... Good thing we can use the web to search for terms we don't know!

In [5]:
#
#    This implements the list request. There are several versions of the list request that only require email and key.
#    This code sets the default action/requests to list the groups or parameter class descriptors. Having those descriptors 
#    allows one to request the individual (proprietary) 5 digit codes for individual air quality measures by using the
#    param request. Some code in later cells will illustrate those requests.
#
def request_list_info(email_address = None, key = None,
                      endpoint_url = API_REQUEST_URL, 
                      endpoint_action = API_ACTION_LIST_CLASSES, 
                      request_template = AQS_REQUEST_TEMPLATE,
                      headers = None):
    
    #  Make sure we have email and key - at least
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    
    # For the basic request we need an email address and a key
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_list_info()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_list_info()'")

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



In [6]:
#
#   The default should get us a list of the various groups or classes of sensors. These classes are user defined names for clustors of
#   sensors that might be part of a package or default air quality sensing station. We need a class name to start getting down to the
#   a sensor ID. Each sensor type has an ID number. We'll eventually need those ID numbers to be able to request values that come from
#   that specific sensor.
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

response = request_list_info(request_template=request_data)



if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "AIRNOW MAPS",
        "value_represented": "The parameters represented on AirNow maps (88101, 88502, and 44201)"
    },
    {
        "code": "ALL",
        "value_represented": "Select all Parameters Available"
    },
    {
        "code": "AQI POLLUTANTS",
        "value_represented": "Pollutants that have an AQI Defined"
    },
    {
        "code": "CORE_HAPS",
        "value_represented": "Urban Air Toxic Pollutants"
    },
    {
        "code": "CRITERIA",
        "value_represented": "Criteria Pollutants"
    },
    {
        "code": "CSN DART",
        "value_represented": "List of CSN speciation parameters to populate the STI DART tool"
    },
    {
        "code": "FORECAST",
        "value_represented": "Parameters routinely extracted by AirNow (STI)"
    },
    {
        "code": "HAPS",
        "value_represented": "Hazardous Air Pollutants"
    },
    {
        "code": "IMPROVE CARBON",
        "value_represented": "IMPROVE Carbon Parameters"
    }

In [7]:
print(response)

{'Header': [{'status': 'Success', 'request_time': '2024-11-18T20:34:41-05:00', 'url': 'https://aqs.epa.gov/data/api/list/classes?email=djsdavis@uw.edu&key=indigofox86', 'rows': 27}], 'Data': [{'code': 'AIRNOW MAPS', 'value_represented': 'The parameters represented on AirNow maps (88101, 88502, and 44201)'}, {'code': 'ALL', 'value_represented': 'Select all Parameters Available'}, {'code': 'AQI POLLUTANTS', 'value_represented': 'Pollutants that have an AQI Defined'}, {'code': 'CORE_HAPS', 'value_represented': 'Urban Air Toxic Pollutants'}, {'code': 'CRITERIA', 'value_represented': 'Criteria Pollutants'}, {'code': 'CSN DART', 'value_represented': 'List of CSN speciation parameters to populate the STI DART tool'}, {'code': 'FORECAST', 'value_represented': 'Parameters routinely extracted by AirNow (STI)'}, {'code': 'HAPS', 'value_represented': 'Hazardous Air Pollutants'}, {'code': 'IMPROVE CARBON', 'value_represented': 'IMPROVE Carbon Parameters'}, {'code': 'IMPROVE_SPECIATION', 'value_repr

We're interested in getting to something that might be the Air Quality Index (AQI). You see this reported on the news - often around smog values, but also when there is smoke in the sky. The AQI is a complex measure of different gasses and of the particles in the air (dust, dirt, ash ...).

From the list produced by our 'list/Classes' request above, it looks like there is a class of sensors called "AQI POLLUTANTS". Let's try to get a list of those specific sensors and see what we can get from those.


In [8]:
#
#   Once we have a list of the classes or groups of possible sensors, we can find the sensor IDs that make up that class (group)
#   The one that looks to be associated with the Air Quality Index is "AQI POLLUTANTS"
#   We'll use that to make another list request.
#
AQI_PARAM_CLASS = "AQI POLLUTANTS"


In [9]:
#
#   Structure a request to get the sensor IDs associated with the AQI
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['pclass'] = AQI_PARAM_CLASS  # here we specify that we want this 'pclass' or parameter classs

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_PARAMS)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "42101",
        "value_represented": "Carbon monoxide"
    },
    {
        "code": "42401",
        "value_represented": "Sulfur dioxide"
    },
    {
        "code": "42602",
        "value_represented": "Nitrogen dioxide (NO2)"
    },
    {
        "code": "44201",
        "value_represented": "Ozone"
    },
    {
        "code": "81102",
        "value_represented": "PM10 Total 0-10um STP"
    },
    {
        "code": "88101",
        "value_represented": "PM2.5 - Local Conditions"
    },
    {
        "code": "88502",
        "value_represented": "Acceptable PM2.5 AQI & Speciation Mass"
    }
]


We should now have (above) a response containing a set of sensor ID numbers. The list should include the sensor numbers as well as a description or name for each sensor. 

The EPA AQS API has limits on some call parameters. Specifically, when we request data for sensors we can only specify a maximum of 5 different sensor values to return. This means we cannot get all of the Air Quality Index parameters in one request for data. We have to break it up.

What I did below was to break the request into two logical groups, the AQI sensors that sample gasses and the AQI sensors that sample particles in the air.

In [10]:
#
#   Given the set of sensor codes, now we can create a parameter list or 'param' value as defined by the AQS API spec.
#   It turns out that we want all of these measures for AQI, but we need to have two different param constants to get
#   all seven of the code types. We can only have a max of 5 sensors/values request per param.
#
#   Gaseous AQI pollutants CO, SO2, NO2, and O2
AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"
#
#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"
#   
#

Experimenting to see what kinds of values come back from different sensor requests for Memphis.

Note: The [FIPS](https://www.census.gov/library/reference/code-lists/ansi.html) number for the state and county as a 5 digit string. This format, the 5 digit string, is a 'old' format that is still widely used. There are new codes that may eventually be adopted for the US government information systems. But FIPS is currently what the AQS uses, so that's what is in the list as the constant.


In [11]:
#
#  This list request should give us a list of all the monitoring stations in the county specified by the
#  given city selected from the CITY_LOCATIONS dictionary
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY_LOCATIONS['memphis']['fips'][:2]   # the first two digits (characters) of FIPS is the state code
request_data['county'] = CITY_LOCATIONS['memphis']['fips'][2:]  # the last three digits (characters) of FIPS is the county code

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_SITES)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))

[
    {
        "code": "0001",
        "value_represented": null
    },
    {
        "code": "0002",
        "value_represented": null
    },
    {
        "code": "0003",
        "value_represented": null
    },
    {
        "code": "0004",
        "value_represented": null
    },
    {
        "code": "0005",
        "value_represented": null
    },
    {
        "code": "0006",
        "value_represented": null
    },
    {
        "code": "0007",
        "value_represented": null
    },
    {
        "code": "0008",
        "value_represented": null
    },
    {
        "code": "0009",
        "value_represented": null
    },
    {
        "code": "0010",
        "value_represented": null
    },
    {
        "code": "0012",
        "value_represented": null
    },
    {
        "code": "0013",
        "value_represented": null
    },
    {
        "code": "0014",
        "value_represented": null
    },
    {
        "code": "0016",
        "value_represented": "Gas Service Cen

Given our CITY_LOCATIONS constant we can now find which monitoring locations are nearby. One option is to use the county to define the area we're interest in. You can get the EPA to list their monitoring stations by county. You can also get a set of monitoring stations by using a bounding box of latitude, longitude points. For this example, we'll use the county approach. There is a bounding box example later in this notebook.


The above response gives us a list of monitoring stations. Each monitoring station has a unique "code" which is a string number, and, sometimes, a description. The description seems to be something about where the monitoring station is located.


## Making a daily summary request
(from US EPA Air Quality System API Example)
The function below is designed to encapsulate requests to the EPA AQS API. When calling the function one should create/copy a parameter template, then initialize that template with values that won't change with each call. Then on each call simply pass in the parameters that need to change, like date ranges.

Another function below provides an example of extracting values and restructuring the response to make it a little more usable.

In [12]:
#
#    This implements the daily summary request. Daily summary provides a daily summary value for each sensor being requested
#    from the start date to the end date. 
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_summary(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL, 
                          endpoint_action = None, 
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):
    
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]            

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_daily_summary()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_daily_summary()'")
    if not request_template['param']: 
        raise Exception("Must supply param values to call 'request_daily_summary()'")
    if not request_template['begin_date']: 
        raise Exception("Must supply a begin_date to call 'request_daily_summary()'")
    if not request_template['end_date']: 
        raise Exception("Must supply an end_date to call 'request_daily_summary()'")
    # Note we're not validating FIPS fields because not all of the daily summary actions require the FIPS numbers
        
    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



In [13]:
#
#    This is a list of field names - data - that will be extracted from each record
#
EXTRACTION_FIELDS = ['sample_duration','observation_count','arithmetic_mean','aqi']

#
#    The function creates a summary record
def extract_summary_from_response(r=None, fields=EXTRACTION_FIELDS):
    ## the result will be structured around monitoring site, parameter, and then date
    result = dict()
    data = r["Data"]
    for record in data:
        # make sure the record is set up
        site = record['site_number']
        param = record['parameter_code']
        #date = record['date_local']    # this version keeps the respnse value YYYY-
        date = record['date_local'].replace('-','') # this puts it in YYYYMMDD format
        if site not in result:
            result[site] = dict()
            result[site]['local_site_name'] = record['local_site_name']
            result[site]['site_address'] = record['site_address']
            result[site]['state'] = record['state']
            result[site]['county'] = record['county']
            result[site]['city'] = record['city']
            result[site]['pollutant_type'] = dict()
        if param not in result[site]['pollutant_type']:
            result[site]['pollutant_type'][param] = dict()
            result[site]['pollutant_type'][param]['parameter_name'] = record['parameter']
            result[site]['pollutant_type'][param]['units_of_measure'] = record['units_of_measure']
            result[site]['pollutant_type'][param]['method'] = record['method']
            result[site]['pollutant_type'][param]['data'] = dict()
        if date not in result[site]['pollutant_type'][param]['data']:
            result[site]['pollutant_type'][param]['data'][date] = list()
        
        # now extract the specified fields
        extract = dict()
        for k in fields:
            if str(k) in record:
                extract[str(k)] = record[k]
            else:
                # this makes sure we always have the requested fields, even if
                # we have a missing value for a given day/month
                extract[str(k)] = None
        
        # add this extraction to the list for the day
        result[site]['pollutant_type'][param]['data'][date].append(extract)
    
    return result


## Get the Daily Summary Data for all Years in Specified Range for Gaseous

My own code, utilizing some aspects of the US EPA Air Quality System API Example

In [14]:
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_GASEOUS
request_data['state'] = CITY_LOCATIONS['memphis']['fips'][:2]
request_data['county'] = CITY_LOCATIONS['memphis']['fips'][2:]


all_gaseous = {}

for year in range(1961, 2022):

    # GET THE GASEAOUS AQI Responses

    # request daily summary data for the month of July in 2021
    gaseous_aqi = request_summary(request_template=request_data, begin_date=f"{year}0101", end_date=f"{year}1231", endpoint_action=API_ACTION_DAILY_SUMMARY_COUNTY)
    #gaseous_aqi = request_summary(request_template=request_data, begin_date=f"{year}0101", end_date=f"{year}1231", endpoint_action=API_ACTION_DAILY_SUMMARY_COUNTY)
    print(f"Response for the gaseous pollutants for {year}...")
    #print(gaseous_aqi["Header"])
    #
    if gaseous_aqi["Header"][0]['status'] == "Success":
        extract_gaseous_year = extract_summary_from_response(gaseous_aqi)
        all_gaseous[year] = extract_gaseous_year  # Store in dictionary with year as key
        #print(json.dumps(gaseous_aqi['Data'],indent=4)
    elif gaseous_aqi["Header"][0]['status'].startswith("No data "):
        print("Looks like the gaseous response generated no data. You might take a closer look at your request and the response data.")
    else:
        #print(json.dumps(gaseous_aqi,indent=4))
        print(gaseous_aqi["Header"])

Response for the gaseous pollutants for 1961...
Looks like the gaseous response generated no data. You might take a closer look at your request and the response data.
Response for the gaseous pollutants for 1962...
Looks like the gaseous response generated no data. You might take a closer look at your request and the response data.
Response for the gaseous pollutants for 1963...
Looks like the gaseous response generated no data. You might take a closer look at your request and the response data.
Response for the gaseous pollutants for 1964...
Looks like the gaseous response generated no data. You might take a closer look at your request and the response data.
Response for the gaseous pollutants for 1965...
Looks like the gaseous response generated no data. You might take a closer look at your request and the response data.
Response for the gaseous pollutants for 1966...
Looks like the gaseous response generated no data. You might take a closer look at your request and the response data

### Determine how many non-null values there are

In [15]:
# Initialize counters for non-null 'aqi' values and total 'aqi' records
aqi_non_null_count = 0
total_records = 0

# Iterate through the 'all_gaseous' dictionary to count non-null 'aqi' values and total records
for year_data in all_gaseous.values():
    for site_info in year_data.values():
        for pollutant_info in site_info['pollutant_type'].values():
            for daily_data in pollutant_info['data'].values():
                for record in daily_data:
                    total_records += 1  # Increment total count for each 'aqi' record
                    if record.get('aqi') is not None:
                        aqi_non_null_count += 1  # Increment non-null count only if 'aqi' is not None

# Display the total counts
print(f"Total 'aqi' records in all_gaseous: {total_records}")
print(f"Total non-null 'aqi' values in all_gaseous: {aqi_non_null_count}")


Total 'aqi' records in all_gaseous: 466173
Total non-null 'aqi' values in all_gaseous: 276613


## Get the Daily Summary Data for all Years in Specified Range for Particulate Data

My own code, utilizing some aspects of the US EPA Air Quality System API Example

In [16]:

request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_GASEOUS
request_data['state'] = CITY_LOCATIONS['memphis']['fips'][:2]
request_data['county'] = CITY_LOCATIONS['memphis']['fips'][2:]


all_particulate = {}

for year in range(1961, 2022):

    # GET THE PARTICULATE AQI RESPONSES

    request_data['param'] = AQI_PARAMS_PARTICULATES
    # request daily summary data for the month of July in 2021
    particulate_aqi = request_summary(request_template=request_data, begin_date=f"{year}0101", end_date=f"{year}1231", endpoint_action=API_ACTION_DAILY_SUMMARY_COUNTY)
    print(f"Response for the particulate pollutants for {year}...")
    
    #
    if particulate_aqi["Header"][0]['status'] == "Success":
        #print("particulate_aqi header success")
        #print(json.dumps(particulate_aqi['Data'],indent=4))

        extract_particulate_year = extract_summary_from_response(particulate_aqi)
        all_particulate[year] = extract_particulate_year 
    
    elif particulate_aqi["Header"][0]['status'].startswith("No data "):
        print("Looks like the particulate response generated no data. You might take a closer look at your request and the response data.")

    else:
        #print(json.dumps(particulate_aqi,indent=4))
        print(particulate_aqi["Header"])

 # Store in dictionary with year as key

    # print("Summary of particulate extraction ...")
    # print(json.dumps(extract_particulate_year,indent=4))

Response for the particulate pollutants for 1961...
Looks like the particulate response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants for 1962...
Looks like the particulate response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants for 1963...
Looks like the particulate response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants for 1964...
Looks like the particulate response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants for 1965...
Looks like the particulate response generated no data. You might take a closer look at your request and the response data.
Response for the particulate pollutants for 1966...
Looks like the particulate response generated no data. You might take a c

### Determine how many non-null values there are

In [17]:
# Initialize counters for non-null 'aqi' values and total 'aqi' records
aqi_non_null_count_particulate = 0
total_records = 0

# Iterate through the 'all_gaseous' dictionary to count non-null 'aqi' values and total records
for year_data in all_particulate.values():
    for site_info in year_data.values():
        for pollutant_info in site_info['pollutant_type'].values():
            for daily_data in pollutant_info['data'].values():
                for record in daily_data:
                    total_records += 1  # Increment total count for each 'aqi' record
                    if record.get('aqi') is not None:
                        aqi_non_null_count_particulate += 1  # Increment non-null count only if 'aqi' is not None

# Display the total counts
print(f"Total 'aqi' records in all_gaseous: {total_records}")
print(f"Total non-null 'aqi' values in all_gaseous: {aqi_non_null_count_particulate}")

Total 'aqi' records in all_gaseous: 147539
Total non-null 'aqi' values in all_gaseous: 131219


## Create dataframe

My own code

In [18]:
# Flatten extraction data function (unchanged)
def flatten_extraction_data(extraction_data_list):
    flat_data = []
    for extraction_data in extraction_data_list:  # Loop over each dictionary in the list
        for site_code, site_info in extraction_data.items():
            site_name = site_info.get('local_site_name')
            for pollutant_number, pollutant_info in site_info['pollutant_type'].items():
                pollutant_name = pollutant_info.get('parameter_name')
                for local_date, daily_data in pollutant_info['data'].items():
                    for record in daily_data:
                        if record.get('aqi') is not None:
                            flat_data.append({
                                'site_code': site_code,
                                'site_name': site_name,
                                'pollutant_number': pollutant_number,
                                'pollutant_name': pollutant_name,
                                'local_date': local_date,
                                'aqi': record['aqi']
                            })
    return flat_data

# Flatten both all_gaseous and all_particulate data
gaseous_flat = flatten_extraction_data(all_gaseous.values())
particulate_flat = flatten_extraction_data(all_particulate.values())

# Combine into a single DataFrame
combined_flat_data = gaseous_flat + particulate_flat
epa_aqi_df = pd.DataFrame(combined_flat_data)

# Display the resulting DataFrame
print(epa_aqi_df.head())

  site_code site_name pollutant_number  pollutant_name local_date  aqi
0      0019      None            42401  Sulfur dioxide   19670102   29
1      0019      None            42401  Sulfur dioxide   19670102   29
2      0019      None            42401  Sulfur dioxide   19670103   29
3      0019      None            42401  Sulfur dioxide   19670103   29
4      0019      None            42401  Sulfur dioxide   19670105   29


In [19]:
num_rows = len(epa_aqi_df)
print(f"Number of rows in df: {num_rows}")

Number of rows in df: 407832


In [20]:
epa_aqi_df.to_csv(EPA_AQI_OUTPUT_PATH, index=False)